In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import os
import re
from scrape_player_data import request_stats_data, export_salary_data, export_stats_data, request_income_data, check_player_duplicates
from setup_database import MySQLConnector
from collections import Counter

In [3]:
for i in range(2001, 2025):
    print(f'### EXPORTING FOR SEASON {i} ###')
    export_salary_data(i)
    export_stats_data(i)

    dat = pd.read_csv(f'player_season_income/{i}_income_data.csv')
    check_player_duplicates(df = dat)


### EXPORTING FOR SEASON 2001 ###
### EXPORTING FOR SEASON 2002 ###
### EXPORTING FOR SEASON 2003 ###
### EXPORTING FOR SEASON 2004 ###
### EXPORTING FOR SEASON 2005 ###
### EXPORTING FOR SEASON 2006 ###
### EXPORTING FOR SEASON 2007 ###
### EXPORTING FOR SEASON 2008 ###
### EXPORTING FOR SEASON 2009 ###
### EXPORTING FOR SEASON 2010 ###
### EXPORTING FOR SEASON 2011 ###
### EXPORTING FOR SEASON 2012 ###
### EXPORTING FOR SEASON 2013 ###
### EXPORTING FOR SEASON 2014 ###
['Tony Mitchell']
### EXPORTING FOR SEASON 2015 ###
### EXPORTING FOR SEASON 2016 ###
### EXPORTING FOR SEASON 2017 ###
### EXPORTING FOR SEASON 2018 ###
### EXPORTING FOR SEASON 2019 ###
### EXPORTING FOR SEASON 2020 ###
### EXPORTING FOR SEASON 2021 ###
### EXPORTING FOR SEASON 2022 ###
### EXPORTING FOR SEASON 2023 ###
### EXPORTING FOR SEASON 2024 ###


### Data pre-processing

In [4]:
# Removes special characters except space (' ')
def remove_special_characters(text):
    return re.sub(r'[^\w\s]', '', text)

Cleaning income data

In [5]:
dfs = []
for year in range(2001, 2025):
    df = pd.read_csv(f'player_season_income/{year}_income_data.csv')

    # Dropping duplicate tony mitchell
    if year == 2014:
        TM_index = df.index[df['Player'] == "Tony Mitchell"].tolist()
        df.drop(max(TM_index), inplace = True)
        check_player_duplicates(df)

    dfs.append(df)

all_seasons_income = pd.concat(dfs)
all_seasons_income['Player'] = all_seasons_income['Player'].apply(remove_special_characters)

In [6]:
# Assigning a player_id to each unique player in all the data
all_players = pd.unique(all_seasons_income['Player'])
players_dict = {key: None for key in all_players}

for i in range(0, len(players_dict)):
    players_dict[all_players[i]] = i + 1

# Maps unique id
all_seasons_income['player_id'] = all_seasons_income['Player'].map(players_dict)
all_seasons_income.drop(columns = ['Player'], inplace = True)
all_seasons_income.insert(0, 'player_id', all_seasons_income.pop('player_id'))
all_seasons_income.insert(1, 'season', all_seasons_income.pop('season'))

del dfs

Creating players data

In [7]:
# Creating data set with player_ids and player name for all distinct players
all_players_df = pd.DataFrame(all_players, columns=["Player"])
all_players_df['player_id'] = all_players_df['Player'].map(players_dict)
all_players_df.insert(0, 'player_id', all_players_df.pop('player_id'))

Cleaning stats data

In [8]:
# Stats data
dfs = []
for year in range(2001, 2025):
    df = pd.read_csv(f'per_game_stats_data/{year}_player_data.csv')
    dfs.append(df)

all_stats = pd.concat(dfs)
all_stats['Player'] = all_stats['Player'].apply(remove_special_characters)
#print('Left out players: ', sorted(set(all_stats['Player']) - set(all_seasons_income['Player'])))

# Maps unique id and drops rows that didn't get assigned an player_id
all_stats['player_id'] = all_stats['Player'].map(players_dict)
all_stats.dropna(subset=['player_id'], inplace = True)
#all_stats = all_stats.astype({'3P%': 'float64', '2P%': 'float64', 'eFG%': 'float64', 'FG%': 'float64', 'FT%': 'float64'})

In [9]:
# Keeps track of (season, player_id) pairs that played for multiple teams (ie players that played for more than 1 team in 1 season)

multiple_teams_CPK = list(all_stats[all_stats.duplicated(subset = ['Player', 'season'], keep = False)][['season', 'player_id']].groupby(['season', 'player_id']).groups)
#all_stats[(all_stats['player_id'] == 23) & (all_stats['season'] == 2001)]

In [10]:
agg_functions = {key: None for key in all_stats.columns}
del agg_functions['Player']
del agg_functions['season']
agg_functions.update(dict.fromkeys(['Pos', 'player_id', 'Age', 'Tm'], 'first'))
agg_functions.update(dict.fromkeys(['G', 'GS'], 'sum'))
agg_functions.update(dict.fromkeys(['MP', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS'], 'mean'))

In [11]:
all_stats = all_stats.groupby(['Player', 'season']).aggregate(agg_functions).reset_index()

for index, row in all_stats.iterrows():
    if (row['season'], row['player_id']) in multiple_teams_CPK:
        all_stats.loc[index, 'Tm'] = 'TOT' # TOT means played for more than 1 team in a season

In [12]:
# Move player_id and season as lead columns
all_stats.insert(0, 'player_id', all_stats.pop('player_id'))
all_stats.insert(1, 'season', all_stats.pop('season'))

# Adjusting NA's. This allows NAs to uploaded to the MySQL database
#all_stats[all_stats.isna().any(axis=1)]
#all_seasons_income[all_seasons_income.isna().any(axis=1)]
all_stats.fillna(r'\N', inplace = True)
all_stats['player_id'] = all_stats['player_id'].astype('int64')
all_stats.drop(columns = ['Player'], inplace = True)

/var/folders/nc/2fhlmlb174j1xg908_jm3bsw0000gn/T/ipykernel_58247/3964145612.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '\N' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  all_stats.fillna(r'\N', inplace = True)


In [13]:
display(all_stats.head(5), all_stats.shape)
display(all_players_df.head(5), all_players_df.shape)
display(all_seasons_income.head(5), all_seasons_income.shape)

,player_id,season,Pos,Age,Tm,G,GS,MP,FG,FGA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1,2001,PF,37,MIA,82,1,17.2,1.8,4.0,...,0.712,1.3,2.5,3.8,0.5,0.4,0.1,0.5,1.5,4.5
1,2230,2023,SG,23,MIL,35,1,9.9,1.5,3.6,...,1.0,0.2,1.1,1.3,0.6,0.2,0.0,0.3,0.9,4.4
2,2230,2024,SG,24,MIL,56,0,11.0,1.5,3.5,...,0.895,0.2,1.0,1.1,0.5,0.2,0.1,0.2,0.9,4.5
3,2231,2023,SF,19,ATL,72,12,19.5,3.4,7.4,...,0.894,0.5,1.6,2.1,1.0,0.6,0.2,0.6,1.2,8.9
4,2231,2024,SF,20,ATL,20,0,8.6,0.9,3.1,...,1.0,0.1,0.8,0.9,0.3,0.1,0.1,0.4,0.3,2.4


(10658, 30)

,player_id,Player
0,1,AC Green
1,2,AJ Guyton
2,3,Aaron McKie
3,4,Aaron Williams
4,5,Adam Keefe


(2425, 2)

,player_id,season,income,adj_income
0,1,2001,2250000,3981991
1,2,2001,316969,560963
2,3,2001,1818000,3217448
3,4,2001,2250000,3981991
4,5,2001,3390000,5999533


(12198, 4)

In [14]:
# Exporing data:
all_stats.to_csv('all_player_stats.csv', index = False)
all_seasons_income.to_csv('all_season_income.csv', index = False)
all_players_df.to_csv('all_players_data.csv', index = False)

### Upload data to database

In [18]:
mysql_connector = MySQLConnector(host="***", user="***", password="***")
mysql_connector.connect_to_db()
mysql_connector.create_nba_db()
mysql_connector.create_players_table()
mysql_connector.create_income_table()
mysql_connector.create_stats_table()

## Connected to database ##


In [ ]:
# Uploading data to database
mysql_connector.load_data(os.getcwd() + '/all_players_data.csv', 'PLAYERS_TABLE')
mysql_connector.load_data(os.getcwd() + '/all_season_income.csv', 'PER_SEASON_INCOME')
mysql_connector.load_data(os.getcwd() + '/all_player_stats.csv', 'PER_GAME_STATS')